In [9]:
import urllib.request, json, datetime, time,pytz
import requests
from requests.structures import CaseInsensitiveDict
with open('keys.json') as json_file:
    keys = json.load(json_file)
trainkey = keys["trainkey"]
buskey = keys["buskey"]

#CTA bus and train trackers require an API key from the CTA.
#the following links have how to get an API key for bus and train trackers:
#https://www.transitchicago.com/developers/bustracker/
#https://www.transitchicago.com/developers/ttdocs/

#see the keys_sample.json file for proper formatting

In [10]:
#this dict defines the origin.  It can have a train station, bus stop, or both.  It can also have multiple of each.
#the example implementation is Ashland & Lake.
#It will calculate transfers from the Green and Pink lines to other L lines;
#transfers from the 9 and X9 to the Orange and Red Lines;
#and transfers from the 9 and X9 to the 66 Chicago bus.
origin = {"trainstop":["40170"],
         "busstop":["14783","6035"]}

transferpoints = []
transferpoints.append({"name":"State & Lake",
                       "trainstop":["40260","41660"],
                        "busstop":[]})
#this is the transfer at State & Lake.  The L stop and the Red Line stop are separate in the CTA's tracking so both are included.

transferpoints.append({"name":"Ashland Orange Line",
                       "trainstop":["41060"],
                        "busstop":["4164","14476","4163"]})
#this is the transfer between the southbound 9 and X9 and the Orange Line at Ashland.
#commented out since it's not so useful for testing
#the time gap between Ashland & Lake to Ashland Orange line is too long for there to be many Orange Line departures
#that are after the 9/X9 southbound arrives at the Orange line

transferpoints.append({"name":"Ashland & Division & Milwaukee",
                       "trainstop":["40320"],
                        "busstop":["6024","6252","2059","2014","5537","5489"]})

transferpoints.append({"name":"Ashland & Chicago",
                       "trainstop":[],
                        "busstop":["15843","622","556","15842"]})
#this is the transfer between the 9 and X9 and the 66

transferroutes = ["56","70","Blue","Red","Org","66"]

In [27]:
def CTAURL(trainstops, busstops):
    train_url = "http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx?key="+trainkey+"&mapid="+trainstops+"&outputType=JSON"
    busurl = "http://www.ctabustracker.com/bustime/api/v2/getpredictions?key="+buskey+"&stpid="+busstops+"&format=json"
    print(train_url)
    print(busurl)
    #this prints the URL of the API results
    #it is useful for troubleshooting/testing

def fetchCTAtrains(trainstops):
    if trainstops == "":
        return
    number_stops = trainstops.count(",")+1
    if number_stops > 4:
        stop_list = trainstops.split(",")
        trainstops_list = []
        while len(stop_list)>0:
            newlist = ""
            for i in range(0,4):
                if len(stop_list)>0:
                    newlist += stop_list[0]+","
                    stop_list.pop(0)
                i = i+1
            newlist = newlist[0:-1]
            trainstops_list.append(newlist)
    else:
        trainstops_list = [trainstops]
    traininfo_list = []
    for trainlist in trainstops_list:
        trainurl = "http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx?key="+trainkey+"&mapid="+trainlist+"&outputType=JSON"
        with urllib.request.urlopen(trainurl) as url:
            some_train_info = json.loads(url.read().decode())["ctatt"]["eta"]
            traininfo_list = traininfo_list + some_train_info
    traininfo = {"ctatt":{
        "eta":traininfo_list
    }}
    return traininfo
    
    print(train_url)
    with urllib.request.urlopen(train_url) as url:
        train_info = json.loads(url.read().decode())
    return train_info

def fetchCTAbuses(busstops):
    if busstops == "":
        return
    number_stops = busstops.count(",")+1
    if number_stops > 10:
        stop_list = busstops.split(",")
        busstops_list = []
        while len(stop_list)>0:
            newlist = ""
            for i in range(0,10):
                if len(stop_list)>0:
                    newlist += stop_list[0]+","
                    stop_list.pop(0)
                i = i+1
            newlist = newlist[0:-1]
            busstops_list.append(newlist)
    else:
        busstops_list = [busstops]
    businfo_list = []
    for buslist in busstops_list:
        busurl = "http://www.ctabustracker.com/bustime/api/v2/getpredictions?key="+buskey+"&stpid="+buslist+"&format=json"
        with urllib.request.urlopen(busurl) as url:
            some_bus_info = json.loads(url.read().decode())["bustime-response"]["prd"]
            businfo_list = businfo_list + some_bus_info
    bus_info = {"bustime-response":{
        "prd":businfo_list
    }}
    return bus_info

#the above two functions call the CTA APIs

def parseCTAtime(timestamp):
    return datetime.datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S")
    #turns the CTA's timestamp into timezone naive datetime objects

In [12]:
#this combines API calls for the origin and transfer to minimize API calls.
# there's a limit on how many stops can be in one call
# so this won't work for if there are a lot of stops, esp for bus stops.
# for an application like that you'd need to split the bus stops into multiple API calls, and combine the results
# I hope to add that functionality later on.

def combineAPI(start,connect):
    #starts with the origin and list of transfer dicts
    alltrains = ""
    for station in start["trainstop"]:
        alltrains += station+","
        #adds the trainstop ID to a string for using in an API
    alltrains = alltrains[0:-1]#removes the trailing comma
    for stop in connect:
        for station in stop["trainstop"]:
            alltrains+=","+station
            #adds every train stop at every connection point to the string
    #now the same for buses
    allbuses = ""
    for station in start["busstop"]:
        allbuses += station+","
    allbuses = allbuses[0:-1]
    for stop in connect:
        for station in stop["busstop"]:
            allbuses+=","+station
    return alltrains,allbuses #returns strings with each stop, with commas in between

In [13]:
def transfercalc(origin,transferpoints,traindata,busdata):
    #calculates the transfers
    trains = traindata["ctatt"]["eta"]
    origintransit,xfertransit = [],[]
    for train in trains:
        ETA = parseCTAtime(train["arrT"]) - parseCTAtime(train["prdt"])
        #calculates the number of minutes till arrival in a time-object
        minutes_prelim = str(ETA)
        train["ETA"] = int(minutes_prelim[2:4]) #adds the number of minutes until arrival as "ETA" for every train
        #the following adds items to each dict to make terminology between buses and trains consistent
        train["des"] = train["destNm"] 
        train["stpnm"] = train["staNm"]
        train["stpid"] = train["staId"]
        #below the trains will be split into departures from the origin and from the transfer point
        if train["stpid"] in origin["trainstop"]:
            origintransit.append(train)
        else:
            xfertransit.append(train)
    
    if "prd" in busdata["bustime-response"].keys():
        buses = busdata["bustime-response"]["prd"]
    else:
        buses = {}
        print("busdata error")
    for bus in buses:
        #adds "ETA" to match terminology above
        if bus["prdctdn"] == "DUE":
            bus["ETA"] = 0
        elif bus["prdctdn"] == "DLY":
            bus["ETA"] = "DLY"
        else:
            bus["ETA"] = int(bus["prdctdn"])
        bus["rn"]=bus["vid"] #makes terminology consistent
        if bus["stpid"] in origin["busstop"]: #splits departures into origin and transfer as above
            origintransit.append(bus)
        else:
            xfertransit.append(bus)
    
    for run1_origin in origintransit: #for every run departing the origin
        if run1_origin["ETA"]=="DLY": #does not calculate transfers for buses that do not have meaningful departure times
                continue
        for run1_xfer in xfertransit: #for every run that shows up at the transfer
            #if the run from the origin shows up at transfer, and the ETA is sooner at origin (so it's going the right way)
            if run1_origin["rn"] == run1_xfer["rn"] and run1_origin["ETA"] < run1_xfer["ETA"]: 
                xferdict = {}
                xferdict["ETA"] = run1_xfer["ETA"]
                transfers = []
                connectionID = ""
                for xfer in transferpoints: #for every transfer location
                    #if the stop the transfer-transit is in the connection being run
                    if run1_xfer["stpid"] in xfer["trainstop"] or run1_xfer["stpid"] in xfer["busstop"]:
                        xferdict["xfername"] = xfer["name"]
                        connectionID = xfer["trainstop"] + xfer["busstop"] #make connectionID, a string of the stop IDs for that transfer
                for run2 in xfertransit: #for every run at the transfer
                    #if the run is stopping at the relevant transfer point...
                    #...and the run we're transferring to isn't a bus with "DLY" as ETA...
                    #...and the transfer-to vehicle is arriving at the transfer after the transfer-from...
                    #...and we're ignoring transferring to the same route the other direction...
                    if run2["stpid"] in connectionID and run2["ETA"]!="DLY" and run2["ETA"]>=run1_xfer["ETA"] and run2["rt"]!=run1_xfer["rt"]:
                        transfers.append(run2)
                        #add that run to the transfer dict
                xferdict["connections"] = transfers
                run1_origin["transfer"] = xferdict
                #the above adds a dict to runs from the origin that have transfers
                #it has "ETA" which is minutes to the transfer point
                #and "xfername" which is the name of the stop, useful when the stops have different names or long names that are awkward to display
                #and "connections", a list of runs that work for that transfer point
    
    return origintransit

In [14]:
def parseconnections(origintransit,transferpoints,transferroutes):
    transitoutputs = []
    #this takes the transfer data and puts it in format that's easier to human-read
    #you could add conditionals here to get specific sorts of transfers from the data
    for run in origintransit: #for every run leaving the origin
        if "rtdir" in run.keys():
            output = run["rt"]+" "+run["rtdir"]+", "+str(run["ETA"])+" minutes"
        else:
            output = run["rt"]+" to "+run["des"]+", "+str(run["ETA"])+" minutes"
        #above prints the basic data, the route, dest, and ETA for every departure at origin
        xfers = []
        if "transfer" in run.keys(): #if the run has a transfer
            for x in run["transfer"]["connections"]: #for every connection
                if x["rt"] in transferroutes: #if that route is one we care about.  
                #this is where you could add more conditionals for a specific application
                    xferoutput = x["rt"]+" connection at "+run["transfer"]["xfername"]+" to "+x["des"]+", layover "+str(x["ETA"]-run["transfer"]["ETA"])+" minutes"
                    #adds a string with data on the connection
                    xfers.append(xferoutput)
        transitoutputs.append([output,xfers])
    #outputs a list of lists.  Each entry in the list is a list with info on the departure from the origin
    #for every departure, the first item in the list is the departure-from-origin string
    #the second item is a list of connections, if any.
    return transitoutputs

In [107]:
def listoutput(origintransit,transferpoints,transferroutes):
    transitoutputs = []
    #this is a function that produces an output that can easily be displayed via CSS
    if origintransit == None:
        return [],[],[],[],[],[]
    raillines = {
        "Red":"redline",
        "Blue":"blueline",
        "Brn":"brownline",
        "G":"greenline",
        "Org":"orangeline",
        "P":"purpleline",
        "Pink":"pinkline",
        "Y":"yellowline"
    }
    northtrains, southtrains = [],[]
    northbus, southbus, eastbus, westbus = [],[],[],[]
    busdict = {"Northbound":{},
              "Southbound":{},
              "Westbound":{},
              "Eastbound":{}}
    busdictlists = {"Northbound":[],
              "Southbound":[],
              "Westbound":[],
              "Eastbound":[]}
    for run in origintransit: #for every train from the origin
        if run["rt"] in raillines.keys(): #if its route says it's a train
            trainstring = str(run["ETA"])+" minutes to "+run["des"]
            traininfo = [raillines[run["rt"]],trainstring]
            if run["trDr"] == "1": #if it's "north" per the CTA designation
                northtrains.append(traininfo)
            elif run["trDr"] == "5":
                southtrains.append(traininfo)
            if "transfer" in run.keys():
                for connection in run["transfer"]["connections"]:
                    if connection["rt"] in transferroutes:
                        layovertime = str(connection["ETA"] - run["transfer"]["ETA"])
                        transferarray = connectionprinter(connection, layovertime)
                        if run["trDr"]=="1" and transferarray != None:
                            northtrains.append(transferarray)
                        elif run["trDr"]=="5" and transferarray != None:
                            southtrains.append(transferarray)
        else:
            if run["rt"] in busdict[run["rtdir"]].keys():
                busdict[run["rtdir"]][run["rt"]].append(run)
            else:
                busdict[run["rtdir"]][run["rt"]] = [run]

    for direction in busdict:
        for route in busdict[direction]:
            busarray = []
            transferlist = []
            ETAarray = []
            busarray.append(route)
            for run in busdict[direction][route]:
                ETAarray.append(run["ETA"])
                dest = run["des"]
                direction = run["rtdir"]
                if "transfer" in run.keys():
                    for connection in run["transfer"]["connections"]:
                        if connection["rt"] in transferroutes:
                            layovertime = str(connection["ETA"] - run["transfer"]["ETA"])
                            transferarray=connectionprinter(connection, layovertime)
                            transferlist.append(transferarray)
            ETAstring = ""
            for est in ETAarray:
                ETAstring += str(est)+", "
            ETAstring = ETAstring[0:-2]
            busarray.append(ETAstring)
            busarray.append(dest)
            busarray.append(direction)
            if busarray != None and busarray != []:
                busdictlists[direction].append(busarray)
            if transferlist != None and transferlist != []:
                busdictlists[direction].append(transferlist)
    
    return northtrains, southtrains, busdictlists["Northbound"], busdictlists["Southbound"],busdictlists["Westbound"],busdictlists["Eastbound"]

def connectionprinter(connection, layovertime):
    raillines = {
        "Red":"redline",
        "Blue":"blueline",
        "Brn":"brownline",
        "G":"greenline",
        "Org":"orangeline",
        "P":"purpleline",
        "Pink":"pinkline",
        "Y":"yellowline"
    }
    if connection["rt"] in raillines.keys():
        transferstring = "Connection at "+connection["stpnm"]+" towards "+connection["des"]+", layover "+layovertime+" minutes"
        transferarray = [raillines[connection["rt"]],transferstring]
    else:
        transferstring = "Connection at "+connection["stpnm"]+" to "+connection["rt"]+" "+connection["rtdir"] +", layover "+layovertime+" minutes"
        transferarray = ["transit",transferstring]

def listprinter(transit): #for testing
    print("Northbound trains")
    for train in transit[0]:
        print(train[0]+": "+train[1])
    print("\nSouthbound trains")
    for train in transit[1]:
        print(train[0]+": "+train[1])
    print("\nNorthbound Buses:")
    for bus in transit[2]:
        busprinter(bus)
    print("\nSouthbound Buses:")
    for bus in transit[3]:
        busprinter(bus)
    print("\nWestbound Buses:")
    for bus in transit[4]:
        busprinter(bus)
    print("\nEastbound Buses:")
    for bus in transit[5]:
        busprinter(bus)
        
def busprinter(bus):
    if type(bus) == list and len(bus) == 4:
        print(bus[0]+" in "+bus[1]+" minutes to "+bus[2])
    else:
        print(bus)

In [108]:
origin = {"trainstop":["41220"],
         "busstop":["5768","5960"]}

transferpoints = []

transferpoints.append({"name":"North/Clybourn",
                       "trainstop":["40650"],
                        "busstop":[]})

transferpoints.append({"name":"Belmont/Clark/Halsted",
                       "trainstop":[],
                        "busstop":["1921","14915","5966"]})

transferpoints.append({"name":"Addison L",
                       "trainstop":[],
                        "busstop":["1921","14915","5966"]})

stopIDs = combineAPI(origin,transferpoints)
#CTAURL(stopIDs[0],stopIDs[1])
traindata = fetchCTAtrains(stopIDs[0])
busdata = fetchCTAbuses(stopIDs[1])

In [109]:
xferdata = transfercalc(origin,transferpoints,traindata,busdata)
testoutput = listoutput(xferdata,transferpoints,transferroutes)
listprinter(testoutput)

Northbound trains
brownline: 3 minutes to Kimball
redline: 5 minutes to Howard
purpleline: 7 minutes to Linden
brownline: 11 minutes to Kimball
redline: 11 minutes to Howard
redline: 19 minutes to Howard

Southbound trains
brownline: 3 minutes to Loop
redline: 7 minutes to 95th/Dan Ryan
purpleline: 11 minutes to Loop
redline: 11 minutes to 95th/Dan Ryan
brownline: 14 minutes to Loop
purpleline: 19 minutes to Loop

Northbound Buses:
8 in 8, 20 minutes to Waveland/Broadway

Southbound Buses:
8 in 4 minutes to 79th

Westbound Buses:

Eastbound Buses:
